In [1]:
import fastf1
import pandas as pd
from collections import defaultdict

# Enable cache
fastf1.Cache.enable_cache('cache')

# Hardcoded experience years for 2024 grid (debut year -> experience = current year - debut year)
driver_experience_years = {
    "VER": 2015, "PER": 2011, "LEC": 2018, "SAI": 2015,
    "HAM": 2007, "RUS": 2019, "NOR": 2019, "PIA": 2023,
    "ALO": 2001, "STR": 2017, "GAS": 2017, "OCO": 2016,
    "BOT": 2013, "ZHO": 2022, "MAG": 2014, "HUL": 2010,
    "ALB": 2019, "SAR": 2023, "TSU": 2021, "RIC": 2011
}


def season_points_with_metrics(year: int = 2024, verbose: bool = False) -> pd.DataFrame:
    schedule = fastf1.get_event_schedule(year).reset_index(drop=True)
    schedule = schedule[schedule['EventFormat'] == 'conventional'].reset_index(drop=True)

    driver_points = defaultdict(float)
    constructor_points = defaultdict(float)
    driver_wins = defaultdict(int)
    driver_podiums = defaultdict(int)
    driver_dnf = defaultdict(int)
    driver_quali_positions = defaultdict(list)
    driver_info = {}

    all_rows = []

    for _, ev in schedule.iterrows():
        race_name = ev['EventName']

        if verbose:
            print(f"\n=== {race_name} ===")

        # Load race session
        try:
            ses = fastf1.get_session(year, race_name, 'R')
            ses.load()
            res = ses.results.copy()
            res['Points'] = pd.to_numeric(res['Points'], errors='coerce').fillna(0.0)
        except Exception as e:
            print(f"Warning: could not load {race_name}: {e}")
            continue

        # Attempt to load qualifying
        quali_positions = {}
        try:
            q = fastf1.get_session(year, race_name, 'Q')
            q.load()
            qres = q.results.copy()
            for _, row in qres.iterrows():
                dn = int(row['DriverNumber'])
                quali_positions[dn] = int(row['Position']) if not pd.isna(row['Position']) else None
        except Exception:
            pass  # fallback: we’ll use race grid if quali missing

        # Build rows before updating this race’s totals
        for _, row in res.iterrows():
            dn = int(row['DriverNumber'])
            abbr = row['Abbreviation']
            team = row['TeamName']

            driver_info[dn] = (abbr, team)

            # Total races so far (before current one) = len(driver_quali_positions[dn])
            races_so_far = len(driver_quali_positions[dn])

            # Compute metrics
            win_rate = driver_wins[dn] / races_so_far if races_so_far > 0 else 0.0
            podium_rate = driver_podiums[dn] / races_so_far if races_so_far > 0 else 0.0
            avg_quali = sum(driver_quali_positions[dn]) / races_so_far if races_so_far > 0 else None
            exp_years = year - driver_experience_years.get(abbr, year)
            dnf_count = driver_dnf[dn]

            # teammate comparison
            teammate = [d for d, (_, t) in driver_info.items() if t == team and d != dn]
            teammate_points = driver_points.get(teammate[0], 0.0) if teammate else 0.0
            teammate_comparison = driver_points.get(dn, 0.0) - teammate_points

            all_rows.append({
                'DriverNumber': dn,
                'Driver': abbr,
                'Constructor': team,
                'RaceName': race_name,
                'DriverPointsSoFar': driver_points.get(dn, 0.0),
                'ConstructorPointsSoFar': constructor_points.get(team, 0.0),
                'NextRacePosition': int(row['Position']) if not pd.isna(row['Position']) else None,
                'DriverWinRateThisSeason': win_rate,
                'DriverPodiumRate': podium_rate,
                'AverageQualifyingPosition': avg_quali,
                'DriverExperienceYears': exp_years,
                'DNFCountThisSeason': dnf_count,
                'TeammateComparison': teammate_comparison
            })

        # After recording, update season accumulators with THIS race results
        for _, row in res.iterrows():
            dn = int(row['DriverNumber'])
            team = row['TeamName']
            abbr = row['Abbreviation']
            pts = float(row['Points'])

            driver_points[dn] += pts
            constructor_points[team] += pts

            # Wins and podiums
            if row['Position'] == 1:
                driver_wins[dn] += 1
            if row['Position'] in [1, 2, 3]:
                driver_podiums[dn] += 1

            # DNF check (status text in FastF1 is "Finished" for normal, otherwise DNF/DQ)
            if str(row['Status']).lower() not in ["finished", "classified finish"]:
                driver_dnf[dn] += 1

            # Quali
            qpos = quali_positions.get(dn)
            if qpos is not None:
                driver_quali_positions[dn].append(qpos)

    df = pd.DataFrame(all_rows).set_index('DriverNumber').sort_index()
    return df


if __name__ == "__main__":
    df = season_points_with_metrics(2024, verbose=True)
    print(df.head(40))


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info



=== Bahrain Grand Prix ===


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_statu


=== Saudi Arabian Grand Prix ===


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session

...


=== Las Vegas Grand Prix ===


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 63: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver  1: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 81: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 30: Lap timing integrity check failed for 2 lap(s)
core        WARNING 


=== Abu Dhabi Grand Prix ===


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_s

             Driver      Constructor                   RaceName  \
DriverNumber                                                      
1               VER  Red Bull Racing         Bahrain Grand Prix   
1               VER  Red Bull Racing  Emilia Romagna Grand Prix   
1               VER  Red Bull Racing        Japanese Grand Prix   
1               VER  Red Bull Racing      Australian Grand Prix   
1               VER  Red Bull Racing          Monaco Grand Prix   
1               VER  Red Bull Racing        Canadian Grand Prix   
1               VER  Red Bull Racing       Las Vegas Grand Prix   
1               VER  Red Bull Racing         Italian Grand Prix   
1               VER  Red Bull Racing     Mexico City Grand Prix   
1               VER  Red Bull Racing      Azerbaijan Grand Prix   
1               VER  Red Bull Racing           Dutch Grand Prix   
1               VER  Red Bull Racing         Spanish Grand Prix   
1               VER  Red Bull Racing       Hungarian Grand Pri

In [2]:
df.head()

,Driver,Constructor,RaceName,DriverPointsSoFar,ConstructorPointsSoFar,NextRacePosition,DriverWinRateThisSeason,DriverPodiumRate,AverageQualifyingPosition,DriverExperienceYears,DNFCountThisSeason,TeammateComparison
DriverNumber,,,,,,,,,,,,
1,VER,Red Bull Racing,Bahrain Grand Prix,0.0,0.0,1,0.000000,0.000000,NaN,9,0,0.0
1,VER,Red Bull Racing,Emilia Romagna Grand Prix,77.0,141.0,1,0.750000,0.750000,1.0,9,1,13.0
1,VER,Red Bull Racing,Japanese Grand Prix,51.0,97.0,1,0.666667,0.666667,1.0,9,1,5.0
1,VER,Red Bull Racing,Australian Grand Prix,51.0,87.0,19,1.000000,1.000000,1.0,9,0,15.0
1,VER,Red Bull Racing,Monaco Grand Prix,102.0,170.0,6,0.800000,0.800000,1.0,9,1,34.0


In [3]:
df

,Driver,Constructor,RaceName,DriverPointsSoFar,ConstructorPointsSoFar,NextRacePosition,DriverWinRateThisSeason,DriverPodiumRate,AverageQualifyingPosition,DriverExperienceYears,DNFCountThisSeason,TeammateComparison
DriverNumber,,,,,,,,,,,,
1,VER,Red Bull Racing,Bahrain Grand Prix,0.0,0.0,1,0.000000,0.000000,NaN,9,0,0.0
1,VER,Red Bull Racing,Emilia Romagna Grand Prix,77.0,141.0,1,0.750000,0.750000,1.000000,9,1,13.0
1,VER,Red Bull Racing,Japanese Grand Prix,51.0,97.0,1,0.666667,0.666667,1.000000,9,1,5.0
1,VER,Red Bull Racing,Australian Grand Prix,51.0,87.0,19,1.000000,1.000000,1.000000,9,0,15.0
1,VER,Red Bull Racing,Monaco Grand Prix,102.0,170.0,6,0.800000,0.800000,1.000000,9,1,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...
81,PIA,McLaren,Japanese Grand Prix,28.0,55.0,8,0.000000,0.000000,6.333333,1,0,1.0
81,PIA,McLaren,Emilia Romagna Grand Prix,32.0,69.0,4,0.000000,0.000000,6.250000,1,0,-5.0
81,PIA,McLaren,Las Vegas Grand Prix,207.0,452.0,7,0.125000,0.375000,5.312500,1,0,-38.0
